In [1]:
import torchvision
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np
import time
import datetime
import requests
import json
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg, Robot
from SCSCtrl import TTLServo
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit

Succeeded to open the port
Succeeded to change the baudrate


In [2]:
# --- [YOLO TensorRT Engine Loader] ---

class TrtYolo:
    def __init__(self, engine_path, input_shape=(1, 3, 640, 640)):
        self.logger = trt.Logger(trt.Logger.WARNING)
        self.runtime = trt.Runtime(self.logger)

        with open(engine_path, "rb") as f:
            self.engine = self.runtime.deserialize_cuda_engine(f.read())

        self.context = self.engine.create_execution_context()
        self.input_shape = input_shape

        self.inputs = []
        self.outputs = []
        self.bindings = []
        self.stream = cuda.Stream()

        for binding in self.engine:
            size = trt.volume(self.engine.get_binding_shape(binding))
            dtype = trt.nptype(self.engine.get_binding_dtype(binding))
            host_mem = cuda.pagelocked_empty(size, dtype)
            device_mem = cuda.mem_alloc(host_mem.nbytes)

            self.bindings.append(int(device_mem))
            if self.engine.binding_is_input(binding):
                self.inputs.append({"host": host_mem, "device": device_mem})
            else:
                self.outputs.append({"host": host_mem, "device": device_mem})

    def infer(self, img):
        np.copyto(self.inputs[0]["host"], img.ravel())

        cuda.memcpy_htod_async(
            self.inputs[0]["device"], self.inputs[0]["host"], self.stream
        )

        self.context.execute_async_v2(
            bindings=self.bindings, stream_handle=self.stream.handle
        )

        for out in self.outputs:
            cuda.memcpy_dtoh_async(out["host"], out["device"], self.stream)

        self.stream.synchronize()
        return self.outputs

In [3]:
# --- [1. 모델 및 장치 설정] ---
# 라인 트래킹 모델 로드 (ResNet18)
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model.load_state_dict(torch.load('best_steering_model_xy_test.pth'))
device = torch.device('cuda')
model = model.to(device).eval().half()

In [4]:
yolo_trt = TrtYolo("best.engine")

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [5]:
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [6]:
# --- [2. 시퀀스 및 상태 변수] ---
# 주황(orange) 노드는 건너뛰고, 아래 순서대로 감지합니다.
color_ranges = {
    'green':  ([40, 30, 50], [80, 255, 255]),
    'blue':   ([95, 120, 70], [115, 255, 255]),
    'purple': ([120, 70, 70], [155, 255, 255]),
    'red':    ([0, 100, 100], [10, 255, 255]),
    'yellow': ([0, 30, 140], [35, 100, 255])
}

In [7]:
target_sequence = ['green', 'purple', 'blue', 'red', 'yellow']
current_step = 0
is_scanning = False
observation_results = [] # 서버 전송용 데이터 저장 리스트

SERVER_URL = "http://172.20.10.6:8888/agv/upload_observation" # FastAPI 주소

In [8]:
# --- [3. 위젯 설정 (기존 코드 유지)] ---
camera = Camera()
image_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [9]:
robot = Robot()
#TTLServo.servoAngleCtrl(5, 1, 1, 150) # Y축 1도
#TTLServo.servoAngleCtrl(1, 0, 1, 150)
TTLServo.servoAngleCtrl(5, -50, 1, 150) # Y축 (높이)

276

In [10]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.16, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.13, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

FloatSlider(value=0.16, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.13, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.0, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

In [11]:
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

In [12]:
# --- [4. 주요 기능 함수] ---

def send_to_server():
    """모든 정찰 종료 후 서버에 일괄 업로드"""
    cycle_id = datetime.datetime.now().strftime("%Y_%m_%d_%H%M")
    payload = {
        "cycle_id": cycle_id,
        "agv_id": "AGV1",
        "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "observations": observation_results
    }
    
    # 서버로 보낼 이미지 파일 리스트 생성 (로컬 사진 사용)
    files = []
    for res in observation_results:
        node = res['node']
        filename = f"{node}.jpg"
        try:
            files.append(("images", (filename, open(filename, "rb"), "image/jpeg")))
        except FileNotFoundError:
            print(f"Warning: {filename} 파일을 찾을 수 없습니다.")

    try:
        data = {"payload": json.dumps(payload)}
        res = requests.post(SERVER_URL, data=data, files=files)
        print(f"서버 응답: {res.status_code}, {res.json()}")
    except Exception as e:
        print(f"업로드 에러: {e}")
    finally:
        for _, f in files: f[1].close()

In [13]:
def run_yolo_on_local_file(node_name):
    """
    TensorRT YOLO 엔진을 사용해 로컬 이미지 추론
    - 출력 shape 가정 ❌
    - reshape ❌
    - 예외 발생 ❌
    - confidence 값은 raw output의 max 값으로 사용
    """

    img_path = f"{node_name}.jpg"
    img = cv2.imread(img_path)

    if img is None:
        print(f"[YOLO] image not found: {img_path}")
        return {"result": "unknown", "confidence": 0.0}

    # ---------------------------
    # 1. Preprocess (YOLO 입력)
    # ---------------------------
    img_resized = cv2.resize(img, (640, 640))
    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
    img_norm = img_rgb.astype(np.float32) / 255.0
    img_chw = np.transpose(img_norm, (2, 0, 1))
    input_tensor = np.expand_dims(img_chw, axis=0).astype(np.float32)

    # ---------------------------
    # 2. TensorRT inference
    # ---------------------------
    try:
        outputs = yolo_trt.infer(input_tensor)
    except Exception as e:
        print("[YOLO] TensorRT inference error:", e)
        return {"result": "unknown", "confidence": 0.0}

    if not outputs or outputs[0]["host"] is None:
        print("[YOLO] empty output")
        return {"result": "unknown", "confidence": 0.0}

    # ---------------------------
    # 3. Output 처리 (🔥 절대 reshape 안 함)
    # ---------------------------
    raw = outputs[0]["host"]

    if raw.size == 0:
        return {"result": "unknown", "confidence": 0.0}

    # confidence = raw tensor의 최대값
    conf_raw = float(np.max(raw))
    conf = max(0.0, min(conf_raw, 1.0))

    # ---------------------------
    # 4. 결과 매핑 (임시 로직)
    # ---------------------------
    # threshold는 look_right_and_scan()에서 판단
    result = "normal" if conf > 0.5 else "unknown"

    return {
        "result": result,
        "confidence": conf
    }


In [14]:
YOLO_TIMEOUT = 5.0        # 최소 5초 루프
YOLO_THRESHOLD = 0.

def look_right_and_scan():
    global is_scanning, current_step
    is_scanning = True
    robot.stop()
    
    node_name = target_sequence[current_step]
    print(f"[SCAN] {node_name} 시작")

    try:
        # 1. 카메라 들기 (준비)
        TTLServo.servoAngleCtrl(5, 1, 1, 150)
        time.sleep(3.0)

        # 2. 오른쪽 회전
        TTLServo.servoAngleCtrl(1, 80, 1, 150)
        time.sleep(3.0)

        # 3. 카메라 내리기 (촬영 각도)
        TTLServo.servoAngleCtrl(5, -30, 1, 150)
        time.sleep(3.0)

        # 4. YOLO 시작 (타임아웃 구간)
        start_time = time.time()
        last_result = {"result": "unknown", "confidence": 0.0}

        while time.time() - start_time < YOLO_TIMEOUT:
            try:
                yolo_res = run_yolo_on_local_file(node_name)
                if yolo_res is not None:
                    last_result = yolo_res
                    if yolo_res["confidence"] >= YOLO_THRESHOLD:
                        break
            except Exception as e:
                print(f"[ERROR] YOLO 추론 중 오류: {e}")
            time.sleep(0.1)

        # 5. 결과 저장
        observation_results.append({
            "node": node_name,
            "yolo": last_result
        })

    finally:
        # 6~8번 과정: 어떤 에러가 나더라도 '반드시' 실행되어야 하는 복귀 로직
        print(f"[RECOVERY] {node_name} 복귀 시퀀스 시작")
        
        # 6. 카메라 다시 들기 (이동을 위해)
        TTLServo.servoAngleCtrl(5, 1, 1, 150)
        time.sleep(1.2) # 시간을 조금 더 넉넉히 배분

        # 7. 정면 보기
        TTLServo.servoAngleCtrl(1, 0, 1, 150)
        time.sleep(1.2)

        # 8. 주행 각도로 세팅
        TTLServo.servoAngleCtrl(5, -50, 1, 150)
        time.sleep(1.0)

        is_scanning = False
        current_step += 1

In [15]:
def detect_color(image, color_name):
    """특정 ROI 내 색상 비율 감지"""
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower = np.array(color_ranges[color_name][0])
    upper = np.array(color_ranges[color_name][1])
    roi = hsv[160:220, 50:170]
    mask = cv2.inRange(roi, lower, upper)
    mask_ratio = np.sum(mask > 0) / mask.size
    return mask_ratio > 0.40

In [16]:
# --- [5. 메인 실행 함수] ---

angle = 0.0
angle_last = 0.0
frame_count = 0

def execute(change):
    global angle, angle_last, is_scanning, current_step, frame_count
    
    if is_scanning:
        return

    image = change['new']
    frame_count += 1

    # [A] 색상 감지 로직
    if frame_count % 3 == 0:
        target_color = target_sequence[current_step]
        if detect_color(image, target_color):
            if target_color == 'yellow':
                print("Yellow 노드(종료) 도달. 전송 중...")
                robot.stop()
                camera.unobserve(execute, names='value')
                send_to_server()
                return
            
            # 주황 노드는 감지 시퀀스에 없으므로 자동으로 스킵됨
            look_right_and_scan()
            frame_count = 0
            return

    # [B] 라인 트래킹 주행 로직
    xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)

In [17]:
# 실행 시작
execute({'new': camera.value})
camera.observe(execute, names='value')

In [ ]:
# import time
# camera.unobserve(execute, names='value')
# time.sleep(0.1)
# robot.stop()
# camera.stop()